### Association Rules
### Market Basket Analysis using Python 

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
df = pd.read_excel(r'Online_Retail.xlsx')
df.head()

ModuleNotFoundError: No module named 'mlxtend'

In [ ]:
# Removing spaces from the description
df['Description'] = df['Description'].str.strip()
#Dropping rows without invoice number
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
# Removing the credit transaction with invoice numbers containing C
df = df[~df['InvoiceNo'].str.contains('C')]
df

In [ ]:
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum()
          .unstack()
          .reset_index()
          .fillna(0)
          .set_index('InvoiceNo'))
basket

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0    
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

In [ ]:
#Generating frequent item sets that have a support of at least 7%
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

#Generating the rules with their corresponding support, confidence and lift
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

rules.head()

In [ ]:
#Filtering out the values with lift > = 6 and confidence > = 0.8
rules[ (rules['lift'] >= 6) & (rules['confidence'] >= 0.8) ]